In [1]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from datetime import datetime, timedelta
from PIL import Image
import rasterio
from rasterio.warp import reproject, Resampling
from rasterio.enums import Resampling

In [2]:
pathi = '/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/masked_predictions/make_it_small/smaller/'
output_kml_path = f'{pathi}png/TS.kml'

In [3]:
inputs = [file for file in getFilelist('/data/Aldhani/eoagritwin/et/Landsat/composites/Brandenburg/2024/', '.tif') if any(f'0{k}' in file for k in range(3,10,1))]

In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS
import os

def reproject_raster_to_4326(input_path, output_path):
    dst_crs = CRS.from_epsg(4326)  # WGS84

    with rasterio.open(input_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds
        )
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open(output_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest
                )


output_dir = f'{pathi}reprojected/'

os.makedirs(output_dir, exist_ok=True)

for in_fp in inputs:
    filename = os.path.basename(in_fp)
    out_fp = os.path.join(output_dir, filename)
    print(f'Reprojecting {filename} to EPSG:4326...')
    reproject_raster_to_4326(in_fp, out_fp)


Reprojecting Landsat_ETA_Brandenburg_2024_03_median.tif to EPSG:4326...
Reprojecting Landsat_ETA_Brandenburg_2024_04_median.tif to EPSG:4326...
Reprojecting Landsat_ETA_Brandenburg_2024_05_median.tif to EPSG:4326...
Reprojecting Landsat_ETA_Brandenburg_2024_06_median.tif to EPSG:4326...
Reprojecting Landsat_ETA_Brandenburg_2024_07_median.tif to EPSG:4326...
Reprojecting Landsat_ETA_Brandenburg_2024_08_median.tif to EPSG:4326...
Reprojecting Landsat_ETA_Brandenburg_2024_09_median.tif to EPSG:4326...
Done.


In [9]:
import rasterio
from rasterio.mask import mask
from shapely.geometry import box
import geopandas as gpd
from pyproj import CRS
from rasterio.coords import BoundingBox

# Bounding box in EPSG:3035
ymin, ymax = 52.69112158007025, 53.026118118098125 # 4468789, 4522900
xmin, xmax = 12.186905421820216, 13.008698861333539# 3291050, 3326422
bbox_crs = CRS.from_epsg(4326)
target_crs = CRS.from_epsg(4326)

# Create GeoDataFrame from bbox
bbox_geom = box(xmin, ymin, xmax, ymax)
bbox_gdf = gpd.GeoDataFrame({"geometry": [bbox_geom]}, crs=bbox_crs)

bboxes = []


inputs = getFilelist(f'{pathi}reprojected/', '.tif')



for file in inputs:
    raster_path = file
    shapefile_path = '/data/Aldhani/eoagritwin/fields/Auxiliary/grid_search/Brandenburg/quick_n_dirty/Fields_polygons.shp'
    output_path = f'{pathi}{file.split("/")[-1]}'

    # Load shapefile
    gdf = gpd.read_file(shapefile_path)

    with rasterio.open(raster_path) as src:
        raster_crs = CRS.from_user_input(src.crs)

        # Reproject to match raster CRS
        if gdf.crs != raster_crs:
            gdf = gdf.to_crs(raster_crs)
        bbox_projected = bbox_gdf.to_crs(raster_crs)

        # Clip shapefile to bounding box
        gdf_clipped = gdf.clip(bbox_projected)
        if gdf_clipped.empty:
            print(f"Empty vector clip for {file}, skipping.")
            continue

        shapes = gdf_clipped.geometry.values

        # Perform mask with crop=True to both clip and mask
        out_image, out_transform = mask(
            dataset=src,
            shapes=shapes,
            crop=True,
            filled=True,
            all_touched=False,
            nodata=src.nodata
        )

        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

    # Save the cropped and masked raster
    with rasterio.open(output_path, "w", **out_meta) as dest:
        dest.write(out_image)

    # Convert bounding box to EPSG:4326 and store
    bbox_wgs84 = bbox_projected.to_crs(target_crs).total_bounds
    subset_bbox = BoundingBox(
        left=bbox_wgs84[0],
        bottom=bbox_wgs84[1],
        right=bbox_wgs84[2],
        top=bbox_wgs84[3]
    )
    bboxes.append(subset_bbox)

In [14]:
from calendar import monthrange
##### insert time dummy
files = getFilelist(pathi, '.tif')
times = []
for file in files:

    date_str = file.split('Brandenburg_')[-1].split('_median')[0]
    year, month = map(int, date_str.split("_"))

    # First day of month
    start = datetime(year, month, 1, 0, 0, 0)

    # Last day of month
    last_day = monthrange(year, month)[1]
    end = datetime(year, month, last_day, 23, 59, 59)

    # Convert to ISO 8601 format with Zulu time (UTC)
    start_str = start.strftime('%Y-%m-%dT%H:%M:%SZ')
    end_str = end.strftime('%Y-%m-%dT%H:%M:%SZ')

    times.append((start_str, end_str))

In [15]:
def parse_qgis_colormap(file_path):
    color_ramp = []
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('#') or line.startswith('INTERPOLATION'):
                continue
            parts = line.strip().split(',')
            value = float(parts[0])
            r, g, b = map(int, parts[1:4])
            hex_color = '#{:02x}{:02x}{:02x}'.format(r, g, b)
            color_ramp.append((value, hex_color))
    return color_ramp

color_ramp = parse_qgis_colormap('/data/Aldhani/eoagritwin/et/Auxiliary/color_for_kml.txt')

# Quick funktion to convert to split hex into RGBA colors, as needed later
def hex_to_rgba(hex_color, alpha=255):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4)) + (alpha,)

def map_value_to_color(value):
    if value == 0:
        return (0, 0, 0, 0)  # Fully transparent for nodata
    for upper_bound, hex_color in color_ramp:
        if value <= upper_bound:
            return hex_to_rgba(hex_color)
    return (0, 0, 0, 0)  # Transparent for out-of-range values


png_files = []


for tiff_file in files:
    print(f"Generating PNG from file {tiff_file}")
    png_path = f'{pathi}png/{tiff_file.split('/')[-1].split('.tif')[0]}.png'

    with rasterio.open(tiff_file) as src:
        band1 = src.read(1)

    # Map values to RGBA
    flat_band = band1.flatten()
    colors = np.array([map_value_to_color(val) for val in flat_band], dtype=np.uint8)
    rgba_image = colors.reshape((band1.shape[0], band1.shape[1], 4))


    # Convert numpy RGBA array to PIL Image and save
    im = Image.fromarray(rgba_image, mode="RGBA")
    im.save(png_path)

    print(f"PNG file {png_path} generated successfully")
    png_files.append(png_path)

Generating PNG from file /data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/masked_predictions/make_it_small/smaller/Landsat_ETA_Brandenburg_2024_03_median.tif
PNG file /data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/masked_predictions/make_it_small/smaller/png/Landsat_ETA_Brandenburg_2024_03_median.png generated successfully
Generating PNG from file /data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/masked_predictions/make_it_small/smaller/Landsat_ETA_Brandenburg_2024_04_median.tif
PNG file /data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/masked_predictions/make_it_small/smaller/png/Landsat_ETA_Brandenburg_2024_04_median.png generated successfully
Generating PNG from file /data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/masked_predictions/make_it_small/smaller/Landsat_ETA_Brandenburg_2024_05_median.tif
PNG file /data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/masked_pre

In [16]:
from xml.etree.ElementTree import Element, SubElement, tostring
from xml.dom.minidom import parseString

kml_files = []

kml_start = """<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://earth.google.com/kml/2.0">
<Document>"""

kml_end = """</Document>
</kml>"""

filled_kml = ""

for png_filename, (west, south, east, north), (begin, end) in zip(png_files, bboxes, times):    
    kml_template = """
 <Folder>
    <name>Raster visibility</name>
    <TimeSpan><begin>{begin}</begin><end>{end}</end></TimeSpan>
    <Folder>
      <name>Raster</name>
      <GroundOverlay>
          <name>Raster data</name>
          <LatLonBox>
            <north>{north}</north>
            <south>{south}</south>
            <west>{west}</west>
            <east>{east}</east>
          </LatLonBox>
          <Icon>
            <href>{href}</href>
          </Icon>
      </GroundOverlay>
    </Folder>
  </Folder>"""
    filled_kml += kml_template.format(
        begin=begin,
        end=end,
        north=north,
        south=south,
        west=west,
        east=east,
        href=os.path.basename(png_filename)
    )

filled_kml = f"{kml_start}{filled_kml}{kml_end}"

with open(output_kml_path, 'w') as f:
    f.write(filled_kml)
print(f"KML file {output_kml_path} generated successfully")

KML file /data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/masked_predictions/make_it_small/smaller/png/TS.kml generated successfully


In [17]:
import zipfile

base_name = os.path.splitext(os.path.basename(output_kml_path))[0]
kmz_path = os.path.join(os.path.dirname(output_kml_path), base_name + '.kmz')

with zipfile.ZipFile(kmz_path, 'w', zipfile.ZIP_DEFLATED) as kmz:
    kmz.write(output_kml_path, os.path.basename(output_kml_path))
    for png_path in png_files:    
        kmz.write(png_path, os.path.basename(png_path))

print(f"KMZ created at: {kmz_path}")

KMZ created at: /data/Aldhani/eoagritwin/fields/output/predictions/FORCE/BRANDENBURG/masked_predictions/make_it_small/smaller/png/TS.kmz
